In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as mp
from sklearn.model_selection import train_test_split

In [ ]:
def tratamentoDados(banco):
  #Realiza a leitura do banco no formato csv e guarda em data
  data = pd.read_csv(banco)
  data.tail()

  #Divisão conjunto de treinamento e testes
  data = np.asarray(data)
  dados = np.zeros((len(data), len(data[0]) - 2))
  classes = np.zeros((len(data), len(data[0]) - 2))

  for i in range(len(data)):
    for j in range(2):
      dados[i][j] = data[i][j]
      classes[i][j] = data[i][j + 2]

  #Separação de dados de treino e teste
  X_train, X_test, y_train, y_test = train_test_split(dados, classes, test_size = 0.2, random_state = 42)

  X_train = X_train.astype('float32') #Converte a variável em float
  X_test = X_test.astype('float32') #Converte a variável em float

  print('Número de dados no conjunto de treinamento:', len(X_train)) #Exibindo dados da variável X_train
  print('Número de dados no conjunto de teste:', len(X_test)) #Exibindo dados da variável X_test
  print('Número de classes no conjunto de treinamento:', len(y_train)) #Exibindo dados da variável X_train
  print('Número de classes no conjunto de teste:', len(y_test)) #Exibindo dados da variável X_test

  print(X_train.shape)
  print(X_test.shape)
  print(y_train.shape)
  print(y_test.shape)

  print('---------------------------------------------')

  return X_train, X_test, y_train, y_test

In [ ]:
def sigmoid(soma):
  return 1 / (1 + np.exp(-soma))

def sigmoidDerivada(sig):
  return sig * (1 - sig)

In [ ]:
def treinar(epocas, entradas, saidas, taxaAprendizagem, momento, pesosEntrada, pesosSaida):
  historicoErro = []
  historicoDelta = []
  for i in range(epocas):
    somaEntrada = np.dot(entradas, pesosEntrada)
    camadaOculta = sigmoid(somaEntrada)

    somaSaida = np.dot(camadaOculta, pesosSaida)
    camadaSaida = sigmoid(somaSaida)

    #print(camadaSaida.shape)
    #print(entradas.shape)
    erro = np.empty((len(saidas), 2))

    erro = saidas - camadaSaida

    #print(erro.shape)
    mediaAbsoluta = np.mean(np.abs(erro))
    #print(mediaAbsoluta)
    historicoErro.append(mediaAbsoluta)

    derivadaSaida = sigmoidDerivada(camadaSaida)
    #print(derivadaSaida.shape)
    #print(pesosSaida.shape)
    delta = erro * derivadaSaida
    #historicoDelta.append(delta)

    pesos1Transposta = pesosSaida.T
    deltaSaidaXPeso = delta.dot(pesos1Transposta)
    deltaCamadaOculta = deltaSaidaXPeso * sigmoidDerivada(camadaOculta)

    camadaSaidaTransposta = camadaOculta.T
    pesosNovoSaida = camadaSaidaTransposta.dot(delta)
    pesosSaida = (pesosSaida * momento) + (pesosNovoSaida * taxaAprendizagem)

    camadaOcultaTransposta = entradas.T
    pesosNovoEntrada = camadaOcultaTransposta.dot(deltaCamadaOculta)
    pesosEntrada = (pesosEntrada * momento) + (pesosNovoEntrada * taxaAprendizagem)

  mp.plot(historicoErro)
  mp.title('Progressão do erro')
  mp.xlabel('Épocas')
  mp.ylabel('Erro')

  # mp.plot(historicoDelta)
  # mp.title('Progressão do delta')
  # mp.xlabel('Épocas')
  # mp.ylabel('Delta')

In [ ]:
def teste(X_test, Y_test, pesosEntrada, pesosSaida):
  somaEntrada = np.dot(X_test, pesosEntrada)
  camadaOculta = sigmoid(somaEntrada)

  somaSaida = np.dot(camadaOculta, pesosSaida)
  camadaSaida = sigmoid(somaSaida)

  #print(Y_test)

  count = 0
  a = np.empty((len(camadaSaida), 2))

  for i in range(len(camadaSaida)):
    if (camadaSaida[i][0] > camadaSaida[i][1]):
      a[i] = [1, 0]
    else:
      a[i] = [0, 1]
    #valorCamadaSaida[i][0] == Y_test[i][0] and valorCamadaSaida[i][1] == Y_test[i][1]
    if (a[i][0] == Y_test[i][0] and a[i][1] == Y_test[i][1]):
      count += 1

  #print(a)
  acc = (count / len(camadaSaida)) * 100
  print('Acurácia no conjunto de teste: ' + str(acc) + '%')

In [ ]:
# =============================== Main ===============================================
banco = 'xor_data.csv'

print('---------------------------------------------')
print('------------------- Dados -------------------')
print('---------------------------------------------')

X_train, X_test, Y_train, Y_test = tratamentoDados(banco)

pesosEntrada = np.random.random((2,5))
pesosSaida = np.random.random((5,2))

epocas = 100000
taxaAprendizagem = 0.002
momento = 1

treinar(epocas, X_train, Y_train, taxaAprendizagem, momento, pesosEntrada, pesosSaida)
teste(X_test, Y_test, pesosEntrada, pesosSaida)